In [17]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

In [19]:
dfCrime = pd.read_csv('/Users/austincoffelt/Documents/Rent-Walk-Local/Incidents_2019.csv')
print(dfCrime.shape)
dfCrime.head()

(50070, 8)


,RB Number,Reported Date,Reported Time,Statute/Ordinance Description,Occurred Location,Occurred District,Occurred Block LAT,Occurred Block LON
0,AL22292,01/01/2019,01:14:14,DOMESTIC ASSAULT,4200 N 93rd St,15,NaN,NaN
1,AL22293,01/01/2019,00:18:38,REQUEST TO LEAVE,1600 Jackson St,52,41.254544,-95.937233
2,AL22294,01/01/2019,01:14:18,ASSAULT - MISDEMEANOR - NO WEAPON,3600 S 132nd St,84,41.224585,-96.119592
3,AL22294,01/01/2019,01:18:03,ASSAULT - MISDEMEANOR - NO WEAPON,3600 S 132nd St,84,41.224585,-96.119592
4,AL22294,01/01/2019,01:07:39,ASSAULT - MISDEMEANOR - NO WEAPON,3600 S 132nd St,84,41.224585,-96.119592


In [21]:
list(dfCrime.columns)

['RB Number',
 'Reported Date',
 'Reported Time',
 'Statute/Ordinance Description',
 'Occurred Location',
 'Occurred District',
 'Occurred Block LAT',
 'Occurred Block LON']

In [23]:
sorted(list(dfCrime['Statute/Ordinance Description'].unique()))

['ABUSE VULNERABLE ADULT -EXPLOIT; FRAUD',
 'ABUSE VULNERABLE ADULT, KNOW & INTENT',
 'ANIMAL BITE',
 'ARSON',
 'ARSON, 1ST DEGREE  -ATTEMPT',
 'ARSON, 2ND DEGREE             -ATTEMPT',
 'ARSON, 3RD DEGREE, $1500/MORE -ATTEMPT',
 'ARSON, 3RD DEGREE, $500-$1499 -ATTEMPT',
 'ARSON, 3RD DEGREE, UNDER $500 -ATTEMPT',
 'ASSAULT - FELONY - AGGRAVATED',
 'ASSAULT - FELONY - AGGRAVATED -ATTEMPT',
 'ASSAULT - MISDEMEANOR - NO WEAPON',
 'ASSAULT - MISDEMEANOR - WEAPON',
 'ASSAULT OFFICER',
 'ASSAULT OFFICER - ATTEMPT',
 'ASSAULT OFFICER/HEALTH CARE PROF, 2ND DEGREE',
 'ASSAULT OFFICER/HEALTH CARE PROF, 3RD DEGREE',
 'BAD CHECK-GOODS,SERV  UNDR $500 -ATMPT',
 'BAD CHECK-GOODS,SERVICE     UNDER $500',
 'BAD CHECK-GOODS,SERVICE    $1500-$4999',
 'BAD CHECK-GOODS,SERVICE    $500-$1499',
 'BURGLARY      -ATTEMPT',
 'BURGLARY-FORCIBLE ENTRY',
 'BURGLARY-NO FORCE ENTRY',
 'CHILD ABUSE BY INTENT-NO SER INJURY',
 'CHILD ABUSE BY INTENT-SER INJ-ATTEMPT',
 'CHILD ABUSE BY INTENT-SERIOUS INJURY',
 'CHILD AB

In [25]:
violentCrimes = ['ASSAULT - FELONY - AGGRAVATED', 'ASSAULT - FELONY - AGGRAVATED -ATTEMPT', 'HOMICIDE', 
                 'HOMICIDE - ATTEMPT', 
                'ROBBERY       -ATTEMPT',
 'ROBBERY -FIREARM',
 'ROBBERY -FIREARM              -ATTEMPT',
 'ROBBERY -KNIFE/CUTTING INSTR  -ATTEMPT',
 'ROBBERY -KNIFE/CUTTING INSTRUMENT',
 'ROBBERY -OTHER DANGEROUS WEAPON',
 'ROBBERY -OTHER WEAPON         -ATTEMPT',
 'ROBBERY -STRONG ARM -NO WEAPON',
 'SEXUAL ASSAULT - CONTACT',
 'SEXUAL ASSAULT-ATT PENETRATION',
 'SEXUAL ASSAULT-PENETRATION']
dfCrime = dfCrime[dfCrime['Statute/Ordinance Description'].isin(violentCrimes)]
dfCrime = dfCrime[['Statute/Ordinance Description', 'Occurred Block LON', 'Occurred Block LAT']]
print(dfCrime.shape)
dfCrime.head()

(2064, 3)


,Statute/Ordinance Description,Occurred Block LON,Occurred Block LAT
36,SEXUAL ASSAULT-PENETRATION,-95.980991,41.217306
39,ROBBERY -FIREARM,-96.023979,41.285101
41,ROBBERY -FIREARM,-96.003515,41.317688
44,ROBBERY -FIREARM,-96.023927,41.311931
73,SEXUAL ASSAULT-PENETRATION,NaN,NaN


In [27]:
# make our lon and lat into a point type for spacial join
dfCrime['geometry'] = dfCrime.apply(lambda row: Point(row['Occurred Block LON'], row['Occurred Block LAT']), axis=1)
dfCrime = dfCrime[['Statute/Ordinance Description', 'geometry']]
print(dfCrime.shape)
dfCrime.head()

(2064, 2)


,Statute/Ordinance Description,geometry
36,SEXUAL ASSAULT-PENETRATION,POINT (-95.98099061999993 41.21730601600007)
39,ROBBERY -FIREARM,POINT (-96.02397912599997 41.28510080300003)
41,ROBBERY -FIREARM,POINT (-96.00351484499998 41.31768754000007)
44,ROBBERY -FIREARM,POINT (-96.02392703699996 41.31193075400006)
73,SEXUAL ASSAULT-PENETRATION,POINT (nan nan)


In [29]:
# Read shapefile (automatically loads .shp, .shx, .dbf)
gdf = gpd.read_file("/Users/austincoffelt/Documents/Rent-Walk-Local/cb_2018_31_bg_500k/cb_2018_31_bg_500k.shp")

# View the first 5 rows
print(gdf.shape)
gdf.head()

(1633, 11)


,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,31,055,007513,2,1500000US310550075132,310550075132,2,BG,2601411,0,"POLYGON ((-96.216 41.263, -96.196 41.263, -96...."
1,31,055,004900,5,1500000US310550049005,310550049005,5,BG,254456,0,"POLYGON ((-95.981 41.265, -95.976 41.265, -95...."
2,31,101,000100,1,1500000US311010001001,311010001001,1,BG,419919,0,"POLYGON ((-101.73 41.132, -101.72 41.133, -101..."
3,31,055,006302,3,1500000US310550063023,310550063023,3,BG,646852,0,"POLYGON ((-96.005 41.306, -96.005 41.307, -95...."
4,31,081,969300,4,1500000US310819693004,310819693004,4,BG,1025878,0,"POLYGON ((-98.017 40.868, -98.006 40.868, -98...."


In [31]:
# make sure points match the crs of gdf
gdfPoints = gpd.GeoDataFrame(
    dfCrime, 
    geometry='geometry',
    crs="EPSG:4269"  # WGS84 coordinate reference system
)
gdfPoints.head()

,Statute/Ordinance Description,geometry
36,SEXUAL ASSAULT-PENETRATION,POINT (-95.981 41.217)
39,ROBBERY -FIREARM,POINT (-96.024 41.285)
41,ROBBERY -FIREARM,POINT (-96.004 41.318)
44,ROBBERY -FIREARM,POINT (-96.024 41.312)
73,SEXUAL ASSAULT-PENETRATION,POINT (nan nan)


In [33]:
# Basic spatial join (points within polygons)
result = gpd.sjoin(
    gdfPoints, 
    gdf,
    how="left",       # 'left', 'right', or 'inner'
    predicate="within" # Spatial relationship
)
print(result.shape)
result.head()

(2064, 13)


,Statute/Ordinance Description,geometry,index_right,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER
36,SEXUAL ASSAULT-PENETRATION,POINT (-95.981 41.217),9.0,31,055,007003,2,1500000US310550070032,310550070032,2,BG,1434224.0,0.0
39,ROBBERY -FIREARM,POINT (-96.024 41.285),1245.0,31,055,006504,4,1500000US310550065044,310550065044,4,BG,1403517.0,3336.0
41,ROBBERY -FIREARM,POINT (-96.004 41.318),385.0,31,055,006301,1,1500000US310550063011,310550063011,1,BG,1744266.0,0.0
44,ROBBERY -FIREARM,POINT (-96.024 41.312),1010.0,31,055,006503,3,1500000US310550065033,310550065033,3,BG,1720348.0,0.0
73,SEXUAL ASSAULT-PENETRATION,POINT (nan nan),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
crime = result['AFFGEOID'].value_counts()
crime

AFFGEOID
1500000US310550012003    36
1500000US310550032002    29
1500000US310550052002    28
1500000US310550040001    22
1500000US310550031004    22
                         ..
1500000US310550074462     1
1500000US310550074291     1
1500000US310550036002     1
1500000US310550074341     1
1500000US310550031002     1
Name: count, Length: 259, dtype: int64

In [37]:
crime.to_csv('/Users/austincoffelt/Documents/RW Data/Omaha_violent_crime_by_BG.csv')